# Start Gui

This notebook can be used to start the GUI and test out stuff.

In [1]:
# -*- coding: utf-8 -*-
"""
@author: Fabian Balzer

***
LICENSE:
    Copyright 2021 Fabian Balzer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

     http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
***

Code to open a GUI Dominion randomizer
"""
# %%

import sys
import randomizer_modules as rm
import PyQt5.QtWidgets as QW
from randomizer_modules.main_ui import UIMainWindow


def start_program():
    """A function to include everything needed to start the application"""
    # Check whether there is already a running QApplication (e.g. if running
    # from an IDE). This setup prevents crashes for the next run:
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    app = UIMainWindow()  # creating the instance
    app.show()
    app.activateWindow()
    qapp.exec_()  # Start the Qt event loop



In [26]:
start_program()

In [7]:
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)
df[df.Types.apply(lambda x: "Ally" in x)][["Name", "IsInSupply"] +[col for col in df.columns if "quality" in col]]
type(df["gain_types"].iloc[0])

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [ ]:

type(df["attack_types"].iloc[0])
path = rm.PATH_MAIN.joinpath(df.iloc[0]["ImagePath"])
df.set_index("Name").loc["Young Witch"]["Text"]
import pandas as pd
selection = ["Way", "Ally"]
kingdom = rm.Kingdom(df, rm.get_randomizer_config_options())
kingdom.randomize([])
kingdom.get_csv_representation(), kingdom.special_targets

In [30]:
import numpy as np
def _calculate_total_quality(values):
    # Initialize array with zeros representing 0, 1, 2, 3, 4
    counts = np.zeros(5)
    
    # If there are 5-i values of the same thing, increment the value count of the next one
    # e.g. a list of [1, 1, 1, 1, 2] should be counted as two 2s. because there are four
    # ones, three 2s will be counted as a single 3, and two 3s will yield a 4.
    for i in range(4):
        counts[i] += values.count(i)
        if i == 0:
            continue
        counts[i+1] += counts[i] // (5-i)
    
    # Look where the first nonzero value sits.
    total_quality_value = np.nonzero(counts)[0][-1]
    
    return total_quality_value

qual = "village"
values = kingdom.get_kingdom_df()[qual + "_quality"].to_list()
print(_calculate_total_quality(values), values)
kingdom.get_kingdom_df()[["Name", f"{qual}_quality"]]

NameError: name 'kingdom' is not defined

In [7]:
path = 'C:/Users/fabia/OneDrive/Dokumente/Freizeit/Dominion/Dominion_randomizer/card_pictures/Base/Cellar.jpg'
import sys
import randomizer_modules as rm
df = rm.read_dataframe_from_file(rm.FPATH_CARD_DATA, True)
import numpy as np
from randomizer_modules.base_widgets import CoolButton
import PyQt5.QtCore as QC
from PyQt5.QtCore import Qt
import PyQt5.QtWidgets as QW
import PyQt5.QtGui as QG
from PyQt5.QtGui import QPixmap, QPainter, QImage, QImageReader, QPainterPath, QPolygon, QColor, QIcon
import pandas as pd
import matplotlib.cm as cm

class HorizontalBarWidget(QW.QFrame):
    """A widget to display a horizontal bar with 4 levels, acting as a RangeSlider"""

    clicked = QC.pyqtSignal(int)

    def __init__(self, parent=None):
        super().__init__(parent)
        self._min_value = 0
        self._max_value = 5
        self._value_range = self._max_value - self._min_value

        self._current_lower_value = 1
        self._current_upper_value = 2

        self.setFrameStyle(QW.QFrame.Box | QW.QFrame.Plain)
        self.setLineWidth(10)
        self.clicked.connect(self.handle_click)

    def setValues(self, lower_value: int, upper_value: int):
        assert lower_value <= upper_value
        self._current_lower_value = lower_value
        self._current_upper_value = upper_value
        self.update()

    def getValues(self) -> tuple[int, int]:
        return self._current_lower_value, self._current_upper_value

    def paintEvent(self, event):
        painter = QG.QPainter(self)
        painter.setRenderHint(QG.QPainter.Antialiasing)

        # Draw the outer rectangular frame
        painter.drawRect(0, 0, self.width(), self.height())

        scale_width = int(self.width() / self._value_range)

        painter.setBrush(QC.Qt.gray)
        start = self._current_lower_value * scale_width
        end = self._current_upper_value * scale_width
        painter.drawRect(start, 1, end-start, self.height() - 2)

        # Draw the scale
        scale_height = 5
        scale_start_y = self.height() - scale_height
        for i in range(5):
            x = i * scale_width
            painter.drawLine(x, scale_start_y, x, self.height())

    def mousePressEvent(self, event: QG.QMouseEvent):
        # Calculate the clicked value based on the click position
        click_value = int(event.x())
        self.clicked.emit(click_value)  # Emit the signal with the clicked value

    def mouseMoveEvent(self, event: QG.QMouseEvent):
        click_value = int(event.x())
        self.clicked.emit(click_value)  # Emit the signal with the clicked value

    def handle_click(self, value):
        value_pos = value / self.width() * self._value_range
        new_bound = int(value_pos)
        lower_bound, upper_bound = self.getValues()
        dist_to_lower = value_pos - lower_bound
        dist_to_upper = upper_bound - value_pos
        if dist_to_lower < -0.5:
            self.setValues(new_bound, upper_bound)
        elif dist_to_upper < -0.5:
            self.setValues(lower_bound, new_bound+1)
        elif dist_to_lower < dist_to_upper and dist_to_lower > 0.5:
            self.setValues(new_bound+1, upper_bound)
        elif dist_to_upper > 0.5:
            self.setValues(lower_bound, new_bound)



class CustomRangeSlider(QW.QWidget):
    """A range slider that allows the user to set two values where the
    second one is larger than the first."""

    value_changed = QC.pyqtSignal()

    def __init__(self):
        super().__init__()

        layout = QW.QVBoxLayout()
        layout.setContentsMargins(2, 2, 2, 2)
        layout.setSpacing(0)

        self.slider_max_value = 4
        self.slider_min = self._init_single_slider()
        layout.addWidget(self.slider_min)

        self.slider_max = self._init_single_slider()
        self.slider_max.setTickPosition(QW.QSlider.TicksBelow)
        layout.addWidget(self.slider_max)

        self.labels_widget = QW.QWidget()
        lay = QW.QHBoxLayout(self.labels_widget)
        lay.setContentsMargins(3, 0, 3, 0)
        for tick in range(self.slider_max_value + 1):
            label = QW.QLabel(str(tick))
            label.setAlignment(QC.Qt.AlignCenter)
            lay.addWidget(label)
            if tick < self.slider_max_value:
                lay.addItem(
                    QW.QSpacerItem(
                        0, 0, QW.QSizePolicy.Expanding, QW.QSizePolicy.Minimum
                    )
                )

        layout.addWidget(self.labels_widget)
        layout.addStretch()
        self.setLayout(layout)

    def _init_single_slider(self) -> QW.QSlider():
        slider = QW.QSlider(QC.Qt.Horizontal)
        return slider

class MainWindow(QW.QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Scrollable QGroupBox")
        self.setGeometry(100, 100, 400, 300)

        self.central_widget = QW.QWidget(self)
        self.setCentralWidget(self.central_widget)

        layout = QW.QVBoxLayout(self.central_widget)
        self.button = self.init_button()
        self.val = 0
        self.slider = QW.QSlider(Qt.Horizontal)
        self.slider.setRange(0, 100)  # Set the range of the slider
        self.slider.valueChanged.connect(self.change_text) 
        self.custom_width = 0
        layout.addWidget(self.slider)
        layout.addWidget(self.button)
    
    def init_button(self):
        button = QW.QPushButton()
        button.clicked.connect(self.reset_all)
        return button
    
    @QC.pyqtSlot()
    def reset_all(self):
        self.custom_width +=1
        self.slider.setValue(self.custom_width)
    
    def change_text(self):
        self.val += 1
        self.button.setText(f"asdf {self.val}")




def start_app():
    qapp = QW.QApplication.instance()
    if not qapp:
        qapp = QW.QApplication(sys.argv)
    window = MainWindow()
    window.show()
    window.activateWindow()
    qapp.exec_()
start_app()
